In [22]:
## imports for custom overlay
from pynq import Overlay
## Import for DMA alloc
from pynq import allocate
## import for video driver API support
from pynq.lib.video import *
# import for timing
import time
## CUSTOM OVERLAY
# 
hudson_overlay = Overlay('coordinates3.bit')
# DMA vars
vdma = hudson_overlay.video.axi_vdma
dma = hudson_overlay.redDMA.axi_dma_0
# config axi lite access
axilite = hudson_overlay.redDMA.threshold_accel_0

In [2]:
# send input buffer and read the output
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
#     dma.sendchannel.transfer(shaped_in)
    
#     dma.sendchannel.transfer(shaped_out)
    dma.sendchannel.wait()
#     dma.recvchannel.wait()

In [23]:
# Set HDMI in and out vars
hdmi_in = hudson_overlay.video.hdmi_in
hdmi_out = hudson_overlay.video.hdmi_out
# initialize IO
hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)



In [24]:
hdmi_in.start()
hdmi_out.start()

In [25]:

# in_buffer = allocate(shape=frame.shape, dtype=np.uint8)
# out_buffer = allocate(shape=frame.shape, dtype=np.uint8)
in_buffer = allocate(shape=720*1280*3, dtype=np.uint8)
# out_buffer = allocate(shape=720*1280*3, dtype=np.uint8)

# show on the screen what an in buffer frame looks like
shaped_in = hdmi_in.readframe()
in_buffer[:] = np.ndarray.flatten(shaped_in)
hdmi_out.writeframe(in_buffer)

In [26]:
# set resolution rows and cols via axi lite
axilite.write(0x10, 10) # thresh
axilite.write(0x18, 255) # maxval
axilite.write(0x20, 720) # rows
axilite.write(0x28, 1280) # cols

In [14]:
# point HDMI out to the out buffer
hdmi_out.writeframe(out_buffer)

numframes = 500
start = time.time()

# time 600 frames of processing
for _ in range(numframes):
    # read a frame, allocate a buffer to hold it
    in_buffer[:] = np.ndarray.flatten(hdmi_in.readframe())
#     run_kernel()
    
    
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

    

Frames per second:  11.758913102127169


1 frame per second when shifting the whole frame red in hardware

In [27]:
axilite.register_map

RegisterMap {
  thresh = Register(thresh=10, RESERVED=0),
  maxval = Register(maxval=255, RESERVED=0),
  rows = Register(rows=720),
  cols = Register(cols=1280),
  xOut_V = Register(xOut_V=500, RESERVED=0),
  xOut_V_ctrl = Register(xOut_V_ap_vld=1, RESERVED=0),
  yOut_V = Register(yOut_V=500, RESERVED=0),
  yOut_V_ctrl = Register(yOut_V_ap_vld=1, RESERVED=0)
}

In [28]:
dma.sendchannel.transfer(in_buffer)

x = axilite.read(0x30)
y = axilite.read(0x38)
# dma.sendchannel.wait()
print(x)
print(y)

500
500


In [29]:
dma.recvchannel.transfer(out_buffer)

RuntimeError: DMA channel not idle

In [12]:
dma?

In [21]:
hdmi_in.close()
hdmi_out.close()
